## Read in box scores

In [100]:
import pandas as pd

In [101]:
df = pd.read_csv('nba_games.csv', index_col=0)

In [102]:
df.sort_values(by="date", inplace=True)
df.reset_index(inplace=True, drop=True)

In [103]:
del df['mp.1']
del df["mp_opp.1"]
del df['index_opp']

In [104]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team", group_keys=False).apply(add_target).copy()

C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\897300223.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\897300223.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\897300223.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

In [105]:
df["target"][pd.isnull(df['target'])] = 2

C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\3382810002.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["target"][pd.isnull(df['target'])] = 2
C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\3382810002.py:1: SettingWithCopyWarn

In [106]:
df['target'] = df['target'].astype(int, errors='ignore')

In [107]:
df["won"].value_counts()

won
False    8886
True     8886
Name: count, dtype: int64

In [108]:
nulls = pd.isnull(df)

In [109]:
nulls = nulls.sum()

In [110]:
nulls = nulls[nulls > 0]

In [111]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [112]:
df = df[valid_columns].copy()

## Feature selection to pick stats that matter

In [113]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

In [114]:
model = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(model, n_features_to_select=30, direction='forward', cv=split)

In [115]:
removed_columns = ['season', 'date', 'won', 'target', 'team', 'team_opp']

In [116]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [117]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [118]:
df.head()

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.363636,0.338235,0.366029,0.206897,0.212121,0.395487,0.418605,0.412698,0.654609,...,0.277279,0.554502,0.317647,GSW,0.451923,1.0,2016,2015-10-27,False,0
1,0.0,0.431818,0.500000,0.322967,0.310345,0.378788,0.368171,0.209302,0.253968,0.519253,...,0.160462,0.345972,0.317647,CHI,0.317308,1.0,2016,2015-10-27,False,1
2,0.0,0.409091,0.397059,0.373206,0.241379,0.227273,0.437055,0.348837,0.349206,0.645274,...,0.088575,0.232227,0.329412,CLE,0.298077,0.0,2016,2015-10-27,True,1
3,0.0,0.500000,0.529412,0.377990,0.310345,0.393939,0.356295,0.441860,0.333333,0.893816,...,0.215661,0.530806,0.505882,NOP,0.298077,0.0,2016,2015-10-27,True,1
4,0.0,0.409091,0.323529,0.435407,0.275862,0.348485,0.351544,0.255814,0.222222,0.766628,...,0.019255,0.203791,0.317647,DET,0.403846,0.0,2016,2015-10-27,False,1


In [119]:
sfs.fit(df[selected_columns], df['target'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [120]:
predictors = list(selected_columns[sfs.get_support()])

In [121]:
predictors

['mp',
 'fg%',
 '3p%',
 'orb',
 'ts%',
 'usg%',
 '3p%_max',
 'ft_max',
 'fta_max',
 '+/-_max',
 'drb%_max',
 'trb%_max',
 'tov%_max',
 'usg%_max',
 'mp_opp',
 'fg_opp',
 '3p_opp',
 'ft%_opp',
 'blk_opp',
 'usg%_opp',
 'fga_max_opp',
 '3p_max_opp',
 'ft_max_opp',
 'ft%_max_opp',
 'blk_max_opp',
 'pf_max_opp',
 'pts_max_opp',
 'drb%_max_opp',
 'blk%_max_opp',
 'usg%_max_opp']

In [122]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []

    seasons = sorted(data['season'].unique())

    for i in range(start, len(seasons), step):
        season = seasons[i]

        train = data[data['season'] < season]
        test = data[data['season'] == season]

        model.fit(train[predictors], train['target'])

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index = test.index)

        combined = pd.concat([test['target'], preds], axis=1)
        combined.columns = ['actual', 'prediction']

        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [123]:
predictions = backtest(df, model, predictors)

In [124]:
predictions

,actual,prediction
5250,1,1
5251,1,1
5252,0,0
5253,1,0
5254,0,1
...,...,...
17767,0,0
17768,1,1
17769,0,1
17770,2,1


In [125]:
from sklearn.metrics import accuracy_score

predictions = predictions[predictions['actual'] != 2]
accuracy_score(predictions['actual'], predictions['prediction'])

0.5485110470701249

In [126]:
df.groupby(['home']).apply(lambda x: x[x['won'] == 1].shape[0] / x.shape[0])

C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\1568926828.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(['home']).apply(lambda x: x[x['won'] == 1].shape[0] / x.shape[0])


home
0.0    0.428314
1.0    0.571686
dtype: float64

In [127]:
df['team'] = df['team'].astype('category')
df['team_numeric'] = df['team'].cat.codes
df = df.copy()

C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\2969744475.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['team_numeric'] = df['team'].cat.codes


In [128]:
team_mapping = dict(enumerate(df['team'].cat.categories))

In [129]:
df_rolling = df[list(selected_columns) + ['won', 'team_numeric', 'season']]

In [130]:
def find_team_averages(team):
    rolling = team.rolling(10).mean()
    return rolling

In [131]:
df_rolling = df_rolling.groupby(['team_numeric', 'season'], group_keys=False).apply(find_team_averages)

C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\428079335.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_rolling = df_rolling.groupby(['team_numeric', 'season'], group_keys=False).apply(find_team_averages)


In [132]:
df_rolling['team'] = df_rolling['team_numeric'].map(team_mapping)
del df_rolling['team_numeric']

In [133]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)

In [134]:
df = df.dropna()

In [135]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby('team', group_keys=False).apply(lambda x: shift_col(x, col_name))

df['home_next'] = add_col(df, 'home')
df['team_opp_next'] = add_col(df, 'team_opp')
df['date_next'] = add_col(df, 'date')

C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\1247971135.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return df.groupby('team', group_keys=False).apply(lambda x: shift_col(x, col_name))
C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\1247971135.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('team', group_keys=False).apply(lambda x: shift_col(x, col_name))
C:\Users\haigh\AppData\Local\Temp\ipykernel_21348\1247971135.py:6: FutureWarning: The default of observed=False is depr

In [137]:
df = df.copy()

In [138]:
full = df.merge(
    df[rolling_cols + ['team_opp_next', 'date_next', 'team']],
    left_on=['team', 'date_next'],
    right_on=['team_opp_next', 'date_next']
    )

In [139]:
full

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,won_10_y,season_10_y,team_10_y,team_opp_next_y,team_y
0,0.00,0.477273,0.500000,0.375598,0.379310,0.348485,0.483373,0.441860,0.396825,0.730455,...,0.273427,0.270616,0.478824,0.308654,0.6,0.7,2016.0,TOR,SAC,TOR
1,0.00,0.340909,0.250000,0.413876,0.310345,0.257576,0.509501,0.511628,0.412698,0.827305,...,0.124904,0.404739,0.408235,0.428846,0.2,0.3,2016.0,SAC,TOR,SAC
2,0.50,0.409091,0.455882,0.330144,0.482759,0.515152,0.437055,0.372093,0.412698,0.568261,...,0.153273,0.344076,0.384706,0.319231,0.7,0.5,2016.0,DET,CLE,DET
3,0.25,0.545455,0.544118,0.416268,0.413793,0.454545,0.419240,0.186047,0.142857,0.883314,...,0.276508,0.352607,0.482353,0.316346,0.7,0.6,2016.0,TOR,GSW,TOR
4,0.00,0.340909,0.558824,0.186603,0.206897,0.469697,0.203088,0.139535,0.111111,0.854142,...,0.156739,0.470142,0.391765,0.436538,0.6,0.1,2016.0,NOP,DEN,NOP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15769,0.00,0.545455,0.426471,0.511962,0.448276,0.469697,0.440618,0.372093,0.365079,0.659277,...,0.235173,0.562085,0.552941,0.429808,0.4,0.6,2022.0,GSW,BOS,GSW
15770,0.00,0.477273,0.455882,0.409091,0.517241,0.590909,0.414489,0.255814,0.222222,0.766628,...,0.170603,0.431754,0.522353,0.348077,0.5,0.6,2022.0,BOS,GSW,BOS
15771,0.00,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.242875,0.567773,0.575294,0.394231,0.4,0.7,2022.0,GSW,BOS,GSW
15772,0.00,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.174711,0.438863,0.483529,0.350000,0.5,0.5,2022.0,BOS,GSW,BOS


In [140]:
removed_columns = list(full.columns[full.dtypes == 'object']) + removed_columns

In [143]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [145]:
sfs.fit(full[selected_columns], full['target'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [146]:
predictors = list(selected_columns[sfs.get_support()])

In [148]:
predictions = backtest(full, model, predictors)

In [149]:
accuracy_score(predictions['actual'], predictions['prediction'])

0.6296296296296297